In [1]:
from google.colab import files
files.upload()

Saving config.yaml to config.yaml


{'config.yaml': b'args:\r\n  dataset: "under-tree/prepared-yagpt"\r\n  special_tokens: ["<history>", "<speaker1>", "<speaker2>"]\r\n  max_len: 128\r\n  batch_size: 8\r\nmodel:\r\n  model_name: "AlexWortega/instruct_rugptMedium"\r\n  checkpoint: "under-tree/YAGPT"'}

In [2]:
!pip install omegaconf transformers huggingface_hub ipywidgets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 67.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 17.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 104.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 87.8 MB/s eta 0:00:00
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.9.3-py3-none-any.whl size=144573 sha256=37cc095ab86c1434d0e7c0f323e48f2e46d7312933e88e961f8c4494727f3867
  Stored in directory: /root/.cache/pip/wheels/12/93/dd/1f6a127edc45659556564c5730f6d4e300888f4bca2d4c5a88
Successfully built antlr4-python3-runtime


In [3]:
import omegaconf

params = omegaconf.OmegaConf.load("config.yaml")

In [4]:
from huggingface_hub import login
login()

In [5]:
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM

checkpoint = params.model.checkpoint
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForCausalLM.from_pretrained(checkpoint)

In [ ]:
# Define the pipeline for text generation
text_generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

In [ ]:
# Generate text using the pipeline
generated_text = text_generator("Seed text for generation", max_length=50, num_return_sequences=1)

# Print the generated text
for text in generated_text:
    print(text['generated_text'].strip())

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Seed text for generation of a new baby.


In [ ]:
# history, speaker1, speaker2
hs, s1, s2 = params.args.special_tokens

def format_dialogue(history, question, answer=''):
    return f'{hs}: {history}\n{s1}: {question}\t{s2}: {answer}'

generator_params = dict(
    do_sample=True, 
    top_k=50, 
    top_p=0.9, 
    num_beams=4,
    temperature=1.2,
    no_repeat_ngram_size=2,
    early_stopping=True,
    return_full_text=False
)
# Generate text using the pipeline
def generate_text(context, question):
    prompt = format_dialogue(context, question)
    l = len(prompt)
    mx_len = l + 20
    generated_text = text_generator(prompt, min_length=l, max_length=mx_len, **generator_params)
    return generated_text

answer = generate_text("Привет, как дела?", "Всё путём. Как сам?")
print(answer[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


 хорошо.	ilya: ты знаешь, я хотел бы встретиться с кем-то из моих друзей, когда мы поедем в город, где я живу, и мы должны встретиться на обед в ресторане, который мы будем искать.history: speaker1: я знаю, что ты думаешь, но я не хочу, чтобы ты волновалась из-за этого


In [ ]:
from pprint import pprint

def generate_text(context, question, ans_len=15):
    prompt = format_dialogue(context, question)
    l = len(prompt)
    mx_len = l + ans_len
    generated_text = text_generator(prompt, min_length=l, max_length=mx_len, **generator_params)
    return generated_text[0]['generated_text']

context = ''
question = ''
while True:
  question = input("Your input: ")
  ans = generate_text(context, question)
  print("Answer: ", ans)
  context += f'{question}\t{ans}\n'

Your input: Привет!


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Answer:   что случилось?	ilya: ничего, просто я уезжаю в город, который мне нравится, и я хотела бы попрощаться с тобой, пока ты не вернёшься домой, но мне нужно будет вернуться на работу, так что я
Your input: Хочу капусту


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Answer:  😊	speaker2: ты хочешь, чтобы я была рядом, когда ты уедешь? я могу позвонить тебе, или ты можешь оставить сообщение на моей странице в инстаграме, если ты захочешь.	
shell: я не хочу, что бы ты беспокоилась, потому что мне очень жаль, я позвоню тебе позже, хорошо? ❤️	beautiful_woman: хорошо, спасибо, ладно.history: speaker1: что ты думаешь о том, почему люди так реагируют на твои шутки и шутки других людей, как будто ты что-то скрываешь от них? (например, не рассказываешь им о своих планах на будущее)?	smileyoullie: ну, на самом деле, они просто не понимают, это не очень хорошо. я думаю, у меня очень хороший друг. но я знаю, поэтому я стараюсь быть более открытым и честной с людьми, чем другие люди. и
Your input: Это бред


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [ ]:
tokenizer.special_tokens_map

In [ ]:
example = "history: speaker1: где ты?\tspeaker2: дамбо.\t\nspeaker1: ты видел музей там?\tspeaker2: да, я только что покинул его."



class Generator:
    def __init__(self) -> None:
        self.context = ''
        
        self.params = dict(
            generator_params={
                "min_length": 50,
                "max_new_tokens": 30,
                "min_new_tokens": 10,
                "top_k": 50,
                "top_p": 0.7,
                "do_sample": True,  
                "early_stopping": True,
                "no_repeat_ngram_size": 2,
                "eos_token_id": tokenizer.eos_token_id,
                "pad_token_id": tokenizer.pad_token_id,
                "bos_token_id": tokenizer.bos_token_id,
                "use_cache": True,
                "repetition_penalty": 1.5,  
                "length_penalty": 2,  
                "num_beams": 4,
                "num_return_sequences": 1,
                "return_full_text": False,
                "device": device
            }
        )

        self.generator = pipeline("text-generation", model=model, tokenizer=tokenizer, **self.params['generator_params'])
    
    def gen_prompt(question, context_len=80):
        return f'{hs}: {self.context[:-context_len]}\n{s1}: {question}\t{s2}:'

    def update_context(self, question, answer):
        self.context =   f'{self.context.rstrip()}\t{s1}: {question}\t{s2}: {answer}\n'

    def generate(self, question):
        prompt = self.gen_prompt(question)
        generated_text = self.generator(prompt, skip_special_tokens=True)
        answer = generated_text[0]['generated_text'].strip()
        answer = answer.split('\t')[0]
        self.update_context(question, answer)
        return answer

In [ ]:
gen = Generator()
inp = ''
while inp != 'exit':
    inp = input("Your input: ")
    ans = gen.generate(inp)
    print("Answer: ", ans)